## Cohere hackathon

- Build a multimodal capability abstraction
- Build an example based on that

In [1]:
from dotenv import load_dotenv
from rich import print

load_dotenv("../.env")

import os
# os.environ["LOGURU_LEVEL"] = "DEBUG"

In [2]:
from agents.tools import wikipedia_search, get_current_weather, image_inspector

from agents.llms._cohere import CohereChatCompletion
from agents.tool_executor import need_tool_use

In [3]:
llm = CohereChatCompletion()

In [ ]:
llm.bind_tools([get_current_weather, wikipedia_search, image_inspector])

In [5]:
messages = [
    {
        "role": "system",
        "content": "You are a helpful assistant that has access to tools. Use the tools only if you don't have answers."
    },
    {"role":"user", "content":"how is the weather in London today?"}
]

output = llm.chat_completion(messages)

if need_tool_use(output):
    tool_results = llm.run_tool(output)
    tool_results[0]["role"] = "assistant"

    updated_messages = messages + tool_results
    updated_messages = updated_messages + [{"role":"system", "content": "Do not use any tools now."},
                                           {"role": "user", "content": "Think step by step and answer my question based on the above context."}
                                          ]
    output = llm.chat_completion(updated_messages)

print(output.choices[0].message.content)

The weather in London, as of 1:58 PM, is partly cloudy with a temperature of 20 degrees Celsius or 68 degrees 
Fahrenheit. The humidity is at 49%, and the UV index is 5. There is no precipitation, and the wind speed is mild at
7 km/h or 4 mph. The pressure is at 1029 millibars, with a visibility range of 10 km or 6 miles. So, overall, it's 
a pleasant day with mild temperatures and partly cloudy skies.

In [9]:
image_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg"

messages = [
    {"role": "system", "content": "You are a helpful assistant that has access to tools and use that to help humans."},
    {"role": "user", "content": f"Check this image {image_url} and suggest me a location where I can go in London which looks similar"}
]
output = llm.chat_completion(messages)

if need_tool_use(output):
    tool_results = llm.run_tool(output)
    tool_results[0]["role"] = "assistant"

    updated_messages = messages + tool_results
    updated_messages = updated_messages + [{"role":"system", "content": "Do not use any tools now."},
                                           {"role": "user", "content": "Think step by step and answer my question based on the above context."}
                                          ]
    output = llm.chat_completion(updated_messages)

if need_tool_use(output):
    tool_results = llm.run_tool(output)
    tool_results[0]["role"] = "assistant"

    updated_messages = messages + tool_results
    updated_messages = updated_messages + [{"role":"system", "content": "Do not use any tools now."},
                                           {"role": "user", "content": "Think step by step and answer my question based on the above context."}
                                          ]
    output = llm.chat_completion(updated_messages)
print(output.choices[0].message)

Message(
    role='assistant',
    content='',
    tool_calls=[
        ToolCall(
            id='7276cb5c4efb4ddab825ec1f54a6611c',
            type='function',
            function=FunctionInfo(
                name='image_inspector',
                arguments='{"image_url_or_path": 
"https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gf
p-wisconsin-madison-the-nature-boardwalk.jpg"}'
            )
        ),
        ToolCall(
            id='477d4a853a7046fbacb566ac5869707f',
            type='function',
            function=FunctionInfo(name='wikipedia_search', arguments='{"query": "London parks with boardwalks"}')
        )
    ]
)